In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import ipywidgets
from IPython.display import display, HTML, Markdown
import sqlite3 as sql
import pandas as pd
import tabulate
from contextlib import closing
from notebook_functions import *

############# PARAMETERS #############


#############################################

## FUNCTIONS ##

def load_data(file):
    
    with closing(sql.connect(file)) as con:
        
        q = 'SELECT angle, x, y, z, brightness FROM image ORDER BY angle, x, y, z'
        table = pd.read_sql(q, con)
        
    angle = np.array(table['angle'])
    field_pos = np.array(table[['x', 'y', 'z']])
    brightness = np.array(table['brightness'])
    
    x, y, z = field_pos.T
    nangle = len(np.uniques(angle))
    nx = len(np.uniques(x))
    ny = len(np.uniques(y))
    nz = len(np.uniques(z))
    
    field_pos = field_pos.reshape((nangle, nx, ny, nz, -1), order='F')
    brightness = brightness.reshape((nangle, nx, ny, nz), order='F')
    angles = np.unique(angle)
    
    return brightness, angles, field_pos


def plot():
    
    fig = plt.figure(figsize=(9,9))
    gs = GridSpec(2,1)
    ax1 = fig.add_subplot(211)
    ax2 = fig.add_subplot(212)
    
    image = ax1.imshow()
    ax1.set_xlabel()
    ax1.set_ylabel()
    contour = ax2.imshow()
    plt.tight_layout()
    
    return image, contour

## SCRIPT ##

path_to_file = open_file_dialog()

parameters = dict()
parameters['File'] = path_to_file
             

# display(Markdown('### <center> Notebook parameters </center>'))
display(HTML('<center>' + tabulate.tabulate(parameters.items(), tablefmt='html') + '</center>'))

brightness, angles, field_pos = load_data(path_to_file)
image, contour = plot()

@ipywidgets.interact(a=0)
def update_plot(a):
    
    idx = np.argmin(np.abs(a - angles))
    
    image.set_data(brightness[idx, ...])
    contour.set_data(brightness[idx, ...])
    